In [2]:
import mediapipe as mp
import cv2
import numpy as np

In [3]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np

def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변환
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [5]:
# MediaPipe Pose 설정
pose = mp.solutions.pose.Pose(
    static_image_mode=False,        # 입력 이미지가 정적 이미지인지, 비디오 스트림 프레임인지 설정(False인 경우 비디오 스트림으로 처리)
    model_complexity=1,             # 0,1,2 의 순으로 정확도가 올라가는 대신 속도가 느림
    smooth_landmarks=True,          # 프레임간의 랜드 마크 위치를 부드럽게 처리할 지 여부 결정
    enable_segmentation=True,       # 포즈 검출과 인물 마스크를 생성할지 여부 결정. 배경 제거 등의 용도로 사용
    smooth_segmentation=True,       # 인물 마스크의 경계를 부드럽게 처리
    min_detection_confidence=0.5,   # 사람 검출 최소 신뢰도 임계값
    min_tracking_confidence=0.5     # 포즈 추적의 신뢰도 임계값
)

# 그리기 함수 초기화
mp_drawing = mp.solutions.drawing_utils

#####################################################################################################


def get_pose(pose_landmarks):

    # 왼쪽 어깨(shoulder) 11 // 오른쪽 어깨(shoulder) 12
    left_shoulder, right_shoulder = pose_landmarks.landmark[11], pose_landmarks.landmark[12]
    
    # 왼쪽 팔꿈치(elbow) 13 // 오른쪽 팔꿈치(elbow) 14
    left_elbow, right_elbow = pose_landmarks.landmark[13], pose_landmarks.landmark[14]
    
    # 왼쪽 손목(wrist) 15 // 오른쪽 손목(wrist) 16
    left_wrist, right_wrist = pose_landmarks.landmark[15], pose_landmarks.landmark[16]

    # 왼쪽 손바닥(palm) 19 // 오른쪽 손바닥(palm) 20
    left_palm, right_palm = pose_landmarks.landmark[19], pose_landmarks.landmark[20]

    # 입
    mouth = pose_landmarks.landmark[10]

    # 얼굴
    face = pose_landmarks.landmark[0]

                           # 팔꿈치.x < 어깨.x                    손목.y > 팔꿈치.y             손바닥.y > 팔꿈치.y             손바닥.x < 어깨.x                     손바닥.x > 오른어깨.x 
    b_Next_Level_Left = left_elbow.x < left_shoulder.x and left_wrist.y < left_shoulder.y and left_palm.y < left_elbow.y and left_palm.x < left_shoulder.x and left_palm.x > right_shoulder.x
    # b_Next_Level_Right = right_elbow.x > right_shoulder

    b_victory = left_wrist.y < face.y and right_wrist.y < face.y and left_wrist.x > right_wrist.x
    b_exit = left_wrist.y < left_shoulder.y and right_wrist.y < right_shoulder.y and left_wrist.x < right_wrist.x
    b_armCross = left_wrist.y < left_elbow.y and right_wrist.y < right_elbow.y and left_wrist.x < left_elbow.x and right_wrist.x > right_elbow.x and left_wrist.x < right_wrist.x
    
    # 포즈 판별
    if b_Next_Level_Left:
        return "Next Level"
    elif b_victory:
        return "만세"
    # elif b_exit:
    #     return "엑스"
    # elif b_armCross:
    #     return "팔짱"
    else:
        return "모름"
        

#####################################################################################################


# # 정답을 맞추었는지 판별하는 함수
def is_correct_answer(pose_result):
    return pose_result == "Next Level"  # 예시로 "Next Level"을 정답으로 설정


#####################################################################################################

# 웹캠 캡처
cap = cv2.VideoCapture(0)

# 정답을 맞춘 경우의 좌표를 저장할 리스트
correct_answer_landmarks = []

f_count = 0
max_count = 100
score = 0
answer_already = False
game_over = False
pose_list = ['Next Level']
target_pose = np.random.choice(pose_list)


while cap.isOpened():
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 추정
    results = pose.process(image)

    # 추정된 포즈 정보 확인
    if results.pose_landmarks:

        # 연결된 랜드마크 시각화
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)


#####################################################################################################

       # 정답을 맞췄을 때의 좌표를 저장
        if is_correct_answer(pose_result):  # 정답을 맞춘 조건을 여기에 입력하세요
            for landmark in [results.pose_landmarks.landmark[15],  # 왼쪽 손목
                             results.pose_landmarks.landmark[13],  # 왼쪽 팔꿈치
                             results.pose_landmarks.landmark[11],  # 왼쪽 어깨
                             results.pose_landmarks.landmark[16],  # 오른쪽 손목
                             results.pose_landmarks.landmark[14],  # 오른쪽 팔꿈치
                             results.pose_landmarks.landmark[12]]: # 오른쪽 어깨
                if landmark.visibility > 0.5:  # 랜드마크가 충분히 신뢰할 경우에만 좌표를 저장합니다
                    correct_answer_landmarks.append((landmark.x, landmark.y))

#####################################################################################################

        # 포즈 판별 및 출력
        pose_result = get_pose(results.pose_landmarks)

        image = draw_text(image, str(score), (30, 50), 30, (0, 255, 0)) # 점수
        if game_over:
            image = draw_text(image, "게임 오버...", (image.shape[1] / 2 - 50, 50), 30, (255, 0, 0))
            image = draw_text(image, "잠시 후 종료됩니다.", (image.shape[1] / 2 - 100, 100), 30, (255, 0, 0))
        elif answer_already:
            image = draw_text(image, "정답!", (image.shape[1] / 2 - 25, 50), 30, (255, 255, 255)) # 정답 노출
        else:
            image = draw_text(image, target_pose, (image.shape[1] / 2 - 25, 50), 30, (255, 255, 255)) # 목표 포즈

        if False == answer_already and False == game_over and pose_result == target_pose:
            score += 1
            answer_already = True


    # 세그멘테이션 마스크 추출 및 배경 제거
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # 이미지 출력
    cv2.imshow('MediaPipe Pose', image)

    # 카운팅 처리
    f_count += 1
    if f_count >= max_count:
        if game_over:
            break
        f_count = 0
        if answer_already:
            if max_count > 40:
                max_count -= 5
            target_pose = np.random.choice(pose_list)
            answer_already = False
        else:
            game_over = True
            max_count = 100

    # 키 입력 처리
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break


#####################################################################################################

# 정답을 맞췄을 때의 각 랜드마크의 좌표 출력 (X, Y)
print("정답을 맞췄을 때의 좌표:")
for landmark in correct_answer_landmarks:
    print(landmark)

cap.release()
cv2.destroyAllWindows()

NameError: name 'pose_result' is not defined

In [4]:
# MediaPipe Pose 모델 및 Hands 모델 초기화
pose = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 그리기 유틸리티 초기화
mp_drawing = mp.solutions.drawing_utils

# 웹캠 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # 이미지 읽기
    ret, image = cap.read()
    if not ret:
        break

    # 이미지를 RGB 포맷으로 변환
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Pose와 Hands 모델로 각각의 추정 수행
    results_pose = pose.process(image_rgb)
    results_hands = hands.process(image_rgb)

    # 추정된 포즈 정보 확인 및 시각화
    if results_pose.pose_landmarks:
        mp_drawing.draw_landmarks(image, results_pose.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)

    # 추정된 손 정보 확인 및 시각화
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)

    # 화면에 이미지 출력
    cv2.imshow('MediaPipe Pose & Hands', image)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 자원 해제
cap.release()
cv2.destroyAllWindows()